In [ ]:
print "Attendi..."
import numpy as np
import matplotlib.pyplot as plt
import os
from math import floor
from os import path

# parametri della simulazione
n_traces = 2 # da impostare
n_ues = 12
n_mmw_enbs = 3
time_quantum = 0.0256 # fisso
time_start = 0.0004 # fisso
time_end = 60.0 # da impostare

# percorso di input
in_path = "/home/luca/Scaricati/output/prova/sinr/" # da impostare

# impostare a "" (stringa vuota) per non caricare i dati dal file npz, altrimenti specificare il nome del file npz
in_file = "" # da impostare
load_from_npz = (in_file != "")

# inizializzazione
times = np.arange(time_start, time_end, time_quantum)
mean_sinr = np.zeros((n_ues, n_mmw_enbs, len(times)))
power_sinr = np.zeros((n_ues, n_mmw_enbs, len(times)))
curr_run = 0
print "Finito!"

In [ ]:
print "Attendi..."
if load_from_npz:
    loaded = np.load(in_path + in_file)
    time_quantum = loaded['time_quantum']
    time_start = loaded['time_start']
    time_end = loaded['time_end']
    times = loaded['times']
    mean_sinr = loaded['mean_sinr']
    delta_sinr = loaded['delta_sinr']
    curr_run = n_traces
    print "Finito!"
    print "Dati caricati da " + in_path + in_file
else:
    in_file = os.listdir(in_path)
    print "Finito!"
    if (len(in_file) != n_traces):
        print "Il numero di tracce non corrisponde! Controllare il percorso di input!"
    else:
        print "Trovate " + str(len(in_file)) + " tracce da analizzare!"

In [ ]:
if load_from_npz:
    print "Eseguire la cella 2 per caricare i dati in memoria!"
elif (len(in_file) != n_traces):
    print "Il numero di tracce non corrisponde! Controllare il percorso di input!"
elif (curr_run == n_traces):
    print "Tutte le tracce sono state analizzate!"
else:
    print "Run: " + str(curr_run) 
    print "File di input: " + in_file[curr_run]
    print "Attendi..."
    
    curr_sinr = np.zeros((n_ues, n_mmw_enbs, len(times)))
    
    with open(in_path + in_file[curr_run], 'r') as f: 
        prev_line = ""
        for line in f:
            # elimino duplicati consecutivi
            if (line != prev_line):
                prev_line = '%s' % line
                tokens = line.rstrip().split(' ')
                time = float(tokens[0])
                if (time >= time_start) and (time < time_end):
                    # estraggo le informazioni
                    ue = int(tokens[1]) - 1
                    enb = int(tokens[2]) - 2
                    sinr_lin = pow(10, (float(tokens[3]) / 10))
                    slot = int(floor((time - time_start) / time_quantum))
                    # accumulo i valori della traccia corrente
                    curr_sinr[ue, enb, slot] = sinr_lin
        f.close()
    # accumulo i valori cross traccia
    mean_sinr += np.divide(curr_sinr, n_traces)
    power_sinr += np.divide(np.square(curr_sinr), n_traces)

    curr_run += 1                   
    print "Finito!"
    print "Bisogna eseguire ancora " + str(n_traces - curr_run) + " run!"  

In [ ]:
if load_from_npz:
    print "Eseguire la cella 2 per caricare i dati in memoria!"
elif (len(in_file) != n_traces):
    print "Il numero di tracce non corrisponde! Controllare il percorso di input!"
elif (curr_run != n_traces):
    print "Bisogna eseguire ancora " + str(n_traces - curr_run) + " run!"
else:
    print "Attendi..."
    # calcolo lo scarto quadratico medio con la formula sqrt(E(x^2) - E(x)^2)
    sigma_sinr = np.sqrt(power_sinr - np.square(mean_sinr))
    
    # converto in dB
    mean_sinr = 10 * np.log10(mean_sinr)
    
    # calcolo delta per l'intervallo di confidenza al 95% in dB
    delta_sinr = 10 * np.log10(1.96 * np.divide(sigma_sinr, np.sqrt(n_traces) * np.ones(len(times))))

    print "Finito!"

In [ ]:
if (not load_from_npz) and (len(in_file) != n_traces):
    print "Il numero di tracce non corrisponde! Controllare il percorso di input!"
elif (curr_run != n_traces):
    print "Bisogna eseguire ancora " + str(n_traces - curr_run) + " run!"
else:
     # plot CI
    for i in range(n_ues):
        fig = plt.figure(figsize = (20, 10))
        ax = plt.gca()
        ax.set_xlim(time_start, time_end)
        ax.xaxis.set_ticks(times)
        ax.xaxis.set_ticklabels(times)
        
        # per migliorare la visualizzazione
        plt.subplot(2, 1, 1)
        plt.title("SINR ue" + str(i + 1) + " / enbs")
        plt.xlabel("Time [s]")
        plt.ylabel('SINR [dB]')
        plt.grid(True)
        
        plt.plot(times, mean_sinr[i, 0], lw = 2, color = 'b')
        plt.plot(times, mean_sinr[i, 1], lw = 2, color = 'r')
        plt.plot(times, mean_sinr[i, 2], lw = 2, color = '#F9A602')
        # ''' decommenta per non mostrare i CI
        plt.plot(times, mean_sinr[i, 0] + delta_sinr[i, 0], '--', lw = 1, color = 'b')
        plt.plot(times, mean_sinr[i, 1] + delta_sinr[i, 1], '--', lw = 1, color = 'r')
        plt.plot(times, mean_sinr[i, 2] + delta_sinr[i, 2], '--', lw = 1, color = '#F9A602')
    
        plt.plot(times, mean_sinr[i, 0] - delta_sinr[i, 0], '--', lw = 1, color = 'b')
        plt.plot(times, mean_sinr[i, 1] - delta_sinr[i, 1], '--', lw = 1, color = 'r')
        plt.plot(times, mean_sinr[i, 2] - delta_sinr[i, 2], '--', lw = 1, color = '#F9A602')
        # '''
        plt.legend(['enb2', 'enb3', "enb4", "CI 95% enb2", "CI 95% enb3", "CI 95% enb4"])

        plt.show() 

In [ ]:
# eseguire per salvare i risultati su file npz
if (not load_from_npz) and (len(in_file) != n_traces):
    print "Il numero di tracce non corrisponde! Controllare il percorso di input!"
elif (curr_run != n_traces):
    print "Bisogna eseguire ancora " + str(n_traces - curr_run) + " run!"
else:
    print "Attendi..."
    # percorso di output
    out_path = "/home/luca/Scrivania/TLCnet_prj/ns3-mmwave/output8/npz/" # da impostare
    out_file = "out.npz" # da impostare
    np.savez_compressed(out_path + out_file, time_quantum = time_quantum, \
                                             time_start = time_start, \
                                             time_end = time_end, \
                                             times = times, \
                                             mean_sinr = mean_sinr, delta_sinr = delta_sinr)
    print "Finito!"
    print "Dati salvati in " + out_path + out_file